In [19]:
print("ok")

ok


In [20]:
%pwd

'c:\\'

In [1]:
import os
os.chdir("/../Medical_Chatbot")

In [2]:
%pwd

'c:\\Medical_Chatbot'

In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

c:\Users\siyam\anaconda3\envs\mbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                         glob="*.pdf", 
                         loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [5]:
extracted_data = load_pdf_file(data= 'Data/')

In [14]:
#extracted_data

In [6]:
#split the data into chunks

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    test_chunks = text_splitter.split_documents(extracted_data)
    return test_chunks

In [7]:
test_chunks = text_split(extracted_data)
print(f"Now you have {len(test_chunks)} chunks of data")

Now you have 5859 chunks of data


In [8]:
#test_chunks

In [9]:
from langchain.embeddings import HuggingFaceEmbeddings

In [10]:
#download the embedding model for huggingface
# 
def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [11]:
embeddings = download_huggingface_embeddings()

C:\Users\siyam\AppData\Local\Temp\ipykernel_25480\3998580563.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [12]:
query_result = embeddings.embed_query("What is diabetes?")
print("length",len(query_result))

lenght 384


In [13]:
#query_result

In [70]:
import os

In [71]:
from dotenv import load_dotenv
load_dotenv()

True

In [72]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")


In [18]:
from pinecone.grpc import PineconeGRPC as pinecone
from pinecone import ServerlessSpec
import os

pc = pinecone(api_key=PINECONE_API_KEY)

index_name = "medical-bot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "medical-bot",
    "metric": "cosine",
    "host": "medical-bot-1ssb7tt.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [84]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [85]:
#embedding

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
        documents=test_chunks,
        index_name=index_name,
        embedding=embeddings,
)

In [74]:
#load existing index

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
        index_name=index_name,
        embedding=embeddings
)

In [75]:
docsearch

In [76]:
retrieved = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [77]:
retrieved_docs = retrieved.invoke("What is diabetes?")

In [78]:
retrieved_docs

[Document(id='e64f7185-adb3-4fe1-b8a2-654617b7ee5c', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 542.0, 'page_label': '543', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='begin to fall. A person with diabetes mellitus either does\nnot make enough insulin, or makes insulin that does not\nwork properly. The result is blood sugar that remains\nhigh, a condition called hyperglycemia.\nDiabetes must be diagnosed as early as possible. If\nleft untreated, it can damage or cause failure of the eyes,\nkidneys, nerves, heart, blood vessels, and other body\norgans. Hypoglycemia, or low blood sugar, may also be\ndiscovered through blood sugar testing. Hypoglycemia is'),
 Document(id='70c199fd-cd59-4e49-a555-e61a5b4a56e3', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:

In [79]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [81]:
from langchain.chains import  create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer,say that you"
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{input}"),
    ]
)

In [82]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retrieved, question_answer_chain)